##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Semantic Search with Approximate Nearest Neighbors and Text Embeddings




The steps are:
1. Download sample data.
2. Generate embeddings for the data using a TF-Hub model
3. Build an ANN index for the embeddings
4. Use the index for similarity matching

We use [Apache Beam](https://beam.apache.org/documentation/programming-guide/) to generate the embeddings from the TF-Hub model. 

We also use Spotify's [ANNOY](https://github.com/spotify/annoy) library to build the approximate nearest neighbours index.

Install the required libraries.

In [2]:
!pip install apache_beam
!pip install 'scikit_learn~=0.23.0'  # For gaussian_random_matrix.
!pip install annoy

     |████████████████████████████████| 9.8 MB 6.7 MB/s 
     |████████████████████████████████| 2.3 MB 32.4 MB/s 
     |████████████████████████████████| 62 kB 760 kB/s 
     |████████████████████████████████| 151 kB 59.9 MB/s 
     |████████████████████████████████| 249 kB 50.3 MB/s 
     |████████████████████████████████| 829 kB 35.3 MB/s 
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43512 sha256=41d40c56e83bdb47533ab476724f6510c60e90ec39e8cd53e72128fdad5b98c1
  Stored in directory: /root/.cache/pip/wheels/bc/49/5f/fdb5b9d85055c478213e0158ac122b596816149a02d82e0ab1
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78546 sha256=2f3caa71a1e3340ac34604cc550bcbec67159702bceaca23a6678a6084cf96c9
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=319abd77b2cfa3afee06ec644f6abfd069fd1ed27f

Import the required libraries

In [3]:
import os
import sys
import pickle
from collections import namedtuple
from datetime import datetime
import numpy as np
!pip install apache_beam
import apache_beam as beam
from apache_beam.transforms import util
import tensorflow as tf
import tensorflow_hub as hub
import annoy
from sklearn.random_projection import gaussian_random_matrix

In [4]:
print('TF version: {}'.format(tf.__version__))
print('TF-Hub version: {}'.format(hub.__version__))
print('Apache Beam version: {}'.format(beam.__version__))

TF version: 2.7.0
TF-Hub version: 0.12.0
Apache Beam version: 2.34.0


## 1. Download Sample Data

Indian news headlines dataset from kaggle



In [5]:
! pip install kaggle
!rm -r ~/.kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download therohk/india-headlines-news-dataset
! unzip india-headlines-news-dataset

rm: cannot remove '/root/.kaggle': No such file or directory
 81% 65.0M/80.4M [00:00<00:00, 66.1MB/s]
100% 80.4M/80.4M [00:01<00:00, 82.1MB/s]
Archive:  india-headlines-news-dataset.zip
  inflating: india-news-headlines.csv  


In [6]:
import pandas as pd
news_df = pd.read_csv("india-news-headlines.csv", nrows = 60000)
print("Dataframe shape:", news_df.shape)
pd.set_option('max_columns', None)
news_df.drop("headline_category",axis=1,inplace=True)
print(news_df.head())
news_df.sort_values("headline_text", inplace = True)
news_df.drop_duplicates(subset ="headline_text",keep = False, inplace = True)
news_df.to_csv('raw.tsv', sep = '\t', index=False)

Dataframe shape: (60000, 3)
   publish_date                                      headline_text
0      20010102  Status quo will not be disturbed at Ayodhya; s...
1      20010102                Fissures in Hurriyat over Pak visit
2      20010102              America's unwanted heading for India?
3      20010102                 For bigwigs; it is destination Goa
4      20010102               Extra buses to clear tourist traffic


For simplicity, we only keep the headline text and remove the publication date

In [7]:
!rm -r corpus
!mkdir corpus

with open('corpus/text.txt', 'w') as out_file:
  with open('raw.tsv', 'r') as in_file:
    for line in in_file:
      headline = line.split('\t')[1].strip().strip('"')
      out_file.write(headline+"\n")

rm: cannot remove 'corpus': No such file or directory


In [8]:
!tail corpus/text.txt

iAppliance to rewrite Net access
iBackOffice plans huge call centre base in Karnataka
iCMG releases world's first CORBA Server
iFlex in tie-up with Trintech
iFlex readies bill presentment product
neoIT MarketMaker to power B2B marketplace
omancing the thorn cactii
reak waters soon for Ullal: MLA
riter bags Marathi award
what the SC order really means


## 2. Generate Embeddings for the Data.

In this tutorial, we use the [Neural Network Language Model (NNLM)](https://tfhub.dev/google/nnlm-en-dim128/2) to generate embeddings for the headline data. The sentence embeddings can then be easily used to compute sentence level meaning similarity. We run the embedding generation process using Apache Beam.

### Embedding extraction method

In [9]:
embed_fn = None

def generate_embeddings(text, model_url, random_projection_matrix=None):
  # Beam will run this function in different processes that need to
  # import hub and load embed_fn (if not previously loaded)
  global embed_fn
  if embed_fn is None:
    embed_fn = hub.load(model_url)
  embedding = embed_fn(text).numpy()
  if random_projection_matrix is not None:
    embedding = embedding.dot(random_projection_matrix)
  return text, embedding


### Convert to tf.Example method

In [10]:
def to_tf_example(entries):
  examples = []

  text_list, embedding_list = entries
  for i in range(len(text_list)):
    text = text_list[i]
    embedding = embedding_list[i]

    features = {
        'text': tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[text.encode('utf-8')])),
        'embedding': tf.train.Feature(
            float_list=tf.train.FloatList(value=embedding.tolist()))
    }
  
    example = tf.train.Example(
        features=tf.train.Features(
            feature=features)).SerializeToString(deterministic=True)
  
    examples.append(example)
  
  return examples

### Beam pipeline

In [11]:
def run_hub2emb(args):
  '''Runs the embedding generation pipeline'''

  options = beam.options.pipeline_options.PipelineOptions(**args)
  args = namedtuple("options", args.keys())(*args.values())

  with beam.Pipeline(args.runner, options=options) as pipeline:
    (
        pipeline
        | 'Read sentences from files' >> beam.io.ReadFromText(
            file_pattern=args.data_dir)
        | 'Batch elements' >> util.BatchElements(
            min_batch_size=args.batch_size, max_batch_size=args.batch_size)
        | 'Generate embeddings' >> beam.Map(
            generate_embeddings, args.model_url, args.random_projection_matrix)
        | 'Encode to tf example' >> beam.FlatMap(to_tf_example)
        | 'Write to TFRecords files' >> beam.io.WriteToTFRecord(
            file_path_prefix='{}/emb'.format(args.output_dir),
            file_name_suffix='.tfrecords')
    )

### Generating Projection Weight Matrix
Enbeddings





In [12]:
#testing the paralell processing pipeline

def generate_random_projection_weights(original_dim, projected_dim):
  random_projection_matrix = None
  random_projection_matrix = gaussian_random_matrix(
      n_components=projected_dim, n_features=original_dim).T
  print("A Gaussian random weight matrix was creates with shape of {}".format(random_projection_matrix.shape))
  print('Storing random projection matrix to disk...')
  with open('random_projection_matrix', 'wb') as handle:
    pickle.dump(random_projection_matrix, 
                handle, protocol=pickle.HIGHEST_PROTOCOL)
        
  return random_projection_matrix
model_url = 'https://tfhub.dev/google/nnlm-en-dim128/2'


### Set dimension
Tradeoff between time to find embedding and accuracy of the embeddings

In [13]:
projected_dim = 64

### Run pipeline

In [14]:
import tempfile

output_dir = tempfile.mkdtemp()
original_dim = hub.load(model_url)(['']).shape[1]
random_projection_matrix = None

if projected_dim:
  random_projection_matrix = generate_random_projection_weights(
      original_dim, projected_dim)

args = {
    'job_name': 'hub2emb-{}'.format(datetime.utcnow().strftime('%y%m%d-%H%M%S')),
    'runner': 'DirectRunner',
    'batch_size': 1024,
    'data_dir': 'corpus/*.txt',
    'output_dir': output_dir,
    'model_url': model_url,
    'random_projection_matrix': random_projection_matrix,
}

print("Pipeline args are set.")
args

A Gaussian random weight matrix was creates with shape of (128, 64)
Storing random projection matrix to disk...
Pipeline args are set.


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function gaussian_random_matrix is deprecated; gaussian_random_matrix is deprecated in 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


{'batch_size': 1024,
 'data_dir': 'corpus/*.txt',
 'job_name': 'hub2emb-211205-012529',
 'model_url': 'https://tfhub.dev/google/nnlm-en-dim128/2',
 'output_dir': '/tmp/tmpfq1zn7tb',
 'random_projection_matrix': array([[-2.24099586e-01, -2.06534132e-01,  7.33447876e-02, ...,
         -8.90196586e-02, -2.91140163e-02, -1.30105420e-02],
        [-8.59390127e-02,  1.28023437e-01, -1.86185313e-01, ...,
          1.90947418e-01, -1.78020521e-01,  4.01730520e-02],
        [ 4.79080752e-02,  3.92872251e-02, -1.49889939e-01, ...,
          2.24436412e-01, -7.03432666e-02, -1.08381454e-01],
        ...,
        [-4.05605790e-02,  1.59242704e-01,  2.31003034e-01, ...,
         -1.34397045e-01, -6.02687845e-03,  1.32325604e-01],
        [ 6.77362094e-02,  4.05236309e-02, -1.47368423e-01, ...,
         -7.53350368e-02, -5.33326883e-02, -2.73748982e-04],
        [ 2.30124394e-01,  1.95459228e-01, -2.79128170e-01, ...,
          2.54372855e-02,  2.27407061e-02, -1.74208985e-01]]),
 'runner': 'DirectR

In [15]:
print("Running pipeline...")
%time run_hub2emb(args)
print("Pipeline is done.")

Running pipeline...


        -8.90196586e-02, -2.91140163e-02, -1.30105420e-02],
       [-8.59390127e-02,  1.28023437e-01, -1.86185313e-01, ...,
         1.90947418e-01, -1.78020521e-01,  4.01730520e-02],
       [ 4.79080752e-02,  3.92872251e-02, -1.49889939e-01, ...,
         2.24436412e-01, -7.03432666e-02, -1.08381454e-01],
       ...,
       [-4.05605790e-02,  1.59242704e-01,  2.31003034e-01, ...,
        -1.34397045e-01, -6.02687845e-03,  1.32325604e-01],
       [ 6.77362094e-02,  4.05236309e-02, -1.47368423e-01, ...,
        -7.53350368e-02, -5.33326883e-02, -2.73748982e-04],
       [ 2.30124394e-01,  1.95459228e-01, -2.79128170e-01, ...,
         2.54372855e-02,  2.27407061e-02, -1.74208985e-01]])}


CPU times: user 8.72 s, sys: 5.83 s, total: 14.5 s
Wall time: 8.62 s
Pipeline is done.


In [16]:
!ls {output_dir}

emb-00000-of-00001.tfrecords


Read some of the generated embeddings...

In [17]:
embed_file = os.path.join(output_dir, 'emb-00000-of-00001.tfrecords')
sample = 5

# Create a description of the features.
feature_description = {
    'text': tf.io.FixedLenFeature([], tf.string),
    'embedding': tf.io.FixedLenFeature([projected_dim], tf.float32)
}

def _parse_example(example):
  # Parse the input `tf.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example, feature_description)

dataset = tf.data.TFRecordDataset(embed_file)
for record in dataset.take(sample).map(_parse_example):
  print("{}: {}".format(record['text'].numpy().decode('utf-8'), record['embedding'].numpy()[:10]))


headline_text: [ 0.0355889   0.20153058  0.14327948  0.00831385  0.13658473  0.04474842
  0.4164145   0.06946948 -0.15532742 -0.02356505]
#Defending the Nation: [-3.0419663e-01 -1.4821230e-02 -1.1612993e-01  1.1138162e-01
 -3.3939795e-03  6.3432746e-02  1.5327128e-04  4.9223311e-02
 -2.0585656e-01  1.4699967e-01]
#LNN Helpline not much of a help: [-0.20051254  0.02992163  0.06441832  0.02852312  0.19194146  0.1005758
 -0.00320513  0.28817672  0.07050508 -0.01515995]
$10 mn for Clinton memoirs: [ 0.0130038   0.05794297 -0.1563591  -0.06837318 -0.11934026 -0.02319421
 -0.1556919  -0.37758648 -0.04424743 -0.14084326]
$100 bn bail-out package in offing: [ 0.12070765 -0.10276997  0.08971539  0.16733016 -0.09254742  0.01975778
 -0.02603568 -0.22195488 -0.09979504 -0.10005516]


## 3. Build the ANN Index for the Embeddings

[ANNOY](https://github.com/spotify/annoy) (Approximate Nearest Neighbors Oh Yeah) is a C++ library with Python bindings to search for points in space that are close to a given query point. It also creates large read-only file-based data structures that are mapped into memory. It is built and used by [Spotify](https://www.spotify.com) for music recommendations. If you are interested you can play along with other alternatives to ANNOY such as [NGT](https://github.com/yahoojapan/NGT), [FAISS](https://github.com/facebookresearch/faiss), etc. 

In [18]:
def build_index(embedding_files_pattern, index_filename, vector_length, 
    metric='angular', num_trees=100):
  '''Builds an ANNOY index'''

  annoy_index = annoy.AnnoyIndex(vector_length, metric=metric)
  # Mapping between the item and its identifier in the index
  mapping = {}

  embed_files = tf.io.gfile.glob(embedding_files_pattern)
  num_files = len(embed_files)
  print('Found {} embedding file(s).'.format(num_files))

  item_counter = 0
  for i, embed_file in enumerate(embed_files):
    print('Loading embeddings in file {} of {}...'.format(i+1, num_files))
    dataset = tf.data.TFRecordDataset(embed_file)
    for record in dataset.map(_parse_example):
      text = record['text'].numpy().decode("utf-8")
      embedding = record['embedding'].numpy()
      mapping[item_counter] = text
      annoy_index.add_item(item_counter, embedding)
      item_counter += 1
      if item_counter % 100000 == 0:
        print('{} items loaded to the index'.format(item_counter))

  print('A total of {} items added to the index'.format(item_counter))

  print('Building the index with {} trees...'.format(num_trees))
  annoy_index.build(n_trees=num_trees)
  print('Index is successfully built.')
  
  print('Saving index to disk...')
  annoy_index.save(index_filename)
  print('Index is saved to disk.')
  print("Index file size: {} GB".format(
    round(os.path.getsize(index_filename) / float(1024 ** 3), 2)))
  annoy_index.unload()

  print('Saving mapping to disk...')
  with open(index_filename + '.mapping', 'wb') as handle:
    pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print('Mapping is saved to disk.')
  print("Mapping file size: {} MB".format(
    round(os.path.getsize(index_filename + '.mapping') / float(1024 ** 2), 2)))

In [19]:
embedding_files = "{}/emb-*.tfrecords".format(output_dir)
embedding_dimension = projected_dim
index_filename = "index"

!rm {index_filename}
!rm {index_filename}.mapping

%time build_index(embedding_files, index_filename, embedding_dimension)

rm: cannot remove 'index': No such file or directory
rm: cannot remove 'index.mapping': No such file or directory
Found 1 embedding file(s).
Loading embeddings in file 1 of 1...
A total of 53809 items added to the index
Building the index with 100 trees...
Index is successfully built.
Saving index to disk...
Index is saved to disk.
Index file size: 0.07 GB
Saving mapping to disk...
Mapping is saved to disk.
Mapping file size: 2.18 MB
CPU times: user 31 s, sys: 2.08 s, total: 33.1 s
Wall time: 25.1 s


In [20]:
!ls

corpus	       india-headlines-news-dataset.zip  random_projection_matrix
index	       india-news-headlines.csv		 raw.tsv
index.mapping  kaggle.json			 sample_data


## 4. Use the Index for Similarity Matching
Now we can use the ANN index to find news headlines that are semantically close to an input query.

### Load the index and the mapping files

In [21]:
index = annoy.AnnoyIndex(embedding_dimension)
index.load(index_filename, prefault=True)
print('Annoy index is loaded.')
with open(index_filename + '.mapping', 'rb') as handle:
  mapping = pickle.load(handle)
print('Mapping file is loaded.')


Annoy index is loaded.
Mapping file is loaded.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


### Similarity matching method

In [22]:
def find_similar_items(embedding, num_matches=5):
  '''Finds similar items to a given embedding in the ANN index'''
  ids = index.get_nns_by_vector(
  embedding, num_matches, search_k=-1, include_distances=False)
  items = [mapping[i] for i in ids]
  return items

### Extract embedding from a given query

In [23]:
# Load the TF-Hub model
print("Loading the TF-Hub model...")
%time embed_fn = hub.load(model_url)
print("TF-Hub model is loaded.")

random_projection_matrix = None
if os.path.exists('random_projection_matrix'):
  print("Loading random projection matrix...")
  with open('random_projection_matrix', 'rb') as handle:
    random_projection_matrix = pickle.load(handle)
  print('random projection matrix is loaded.')

def extract_embeddings(query):
  '''Generates the embedding for the query'''
  query_embedding =  embed_fn([query])[0].numpy()
  if random_projection_matrix is not None:
    query_embedding = query_embedding.dot(random_projection_matrix)
  return query_embedding


Loading the TF-Hub model...
CPU times: user 577 ms, sys: 294 ms, total: 872 ms
Wall time: 878 ms
TF-Hub model is loaded.
Loading random projection matrix...
random projection matrix is loaded.


In [24]:
extract_embeddings("Hello Machine Learning!")[:10] #test query string

array([ 0.14128624, -0.17463988,  0.09110139, -0.12706835, -0.16837585,
       -0.03135261,  0.07596967,  0.09212895, -0.05135348, -0.12892351])

### Enter a query to find the most similar items

In [25]:
query = "army of Britian to start helicopter crash probe" #@param {type:"string"}

print("Generating embedding for the query...")
%time query_embedding = extract_embeddings(query)

print("")
print("Finding relevant items in the index...")
%time items = find_similar_items(query_embedding, 20)

print("")
print("Results:")
print("=========")
for item in items:
  print(item)

Generating embedding for the query...
CPU times: user 5.41 ms, sys: 74 µs, total: 5.49 ms
Wall time: 5.37 ms

Finding relevant items in the index...
CPU times: user 2.7 ms, sys: 941 µs, total: 3.64 ms
Wall time: 2.9 ms

Results:
British forces to begin copter crash investigation
Police to buy unmanned plane to spy on naxalites
Spate of burglaries: Police team sent to Mumbai
Army job hopeful killed in stampede
Aziz to visit US to firm up financial package
Plea for CoD probe into boat fire
Anti sabotage jeeps for Punjab police
Case against firm for misuse of machinery
DIG to probe into mysterious killing of trader
Joint panel to probe airport incident
Separate economic offences arm of CBI
Survivor of flyover crash questioned
Oppn fire failed to singe government
US victim of own deeds
Police seize documents from ex-MD of Tata Finance
Concern over transfer of Kandla land to state govt
SC orders probe into dump of hazardous waste
Murder: Cops leave in search of duo
Trade bodies to oppose tr

In [26]:
# to keep notebook up even if network goes down after model runs.
import time
for i in range(10000):
  time.sleep(10)
  print(i)


0
1


KeyboardInterrupt: ignored